<a href="https://colab.research.google.com/github/NotARectangle/Honours2021/blob/dataTest/HuggingFaceCustom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install transformers 

     |████████████████████████████████| 1.8MB 10.7MB/s 
     |████████████████████████████████| 890kB 38.6MB/s 
     |████████████████████████████████| 3.2MB 38.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=65625d0eb2833c6b3e739aa6c26f068f378e78aa073ddeaa1d1da49d21e617d2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 26kB/s 
     |████████████████████████████████| 12.8MB 12.5MB/s 
     |████████████████████████████████| 7.6MB 6.6MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [4]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.0930, 0.9448, 0.9620],
        [0.1581, 0.1999, 0.7891],
        [0.9870, 0.1608, 0.3017],
        [0.9614, 0.2200, 0.9184],
        [0.8447, 0.5814, 0.4762]])


In [5]:
torch.cuda.is_available()

True

In [7]:
!pip install datasets

     |████████████████████████████████| 163kB 10.8MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 
     |████████████████████████████████| 245kB 37.4MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


Fine Tuning with Custom Datasets

In [18]:
from datasets import load_dataset

imdb = load_dataset("imdb")

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


In [19]:
print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [20]:
train = imdb.get("train")

In [23]:
print(train)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [24]:
print(train.features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None)}


In [26]:
print(train.column_names)

['label', 'text']


In [31]:
train_text = train['text']
print(len(train_text))

25000


In [32]:
train.features['label'].names

['neg', 'pos']

In [36]:
print(train[0]["text"])

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [38]:
print(train.num_rows)

25000


In [42]:
def read_imdb_split(splitkey):
  text = []
  labels = []
  subImdb = imdb.get(splitkey)
  for x in subImdb:
    text.append(x["text"])
    labels.append(x["label"])

  return text, labels

train_text, train_labels = read_imdb_split("train")
test_text, test_labels = read_imdb_split("test")

In [44]:
print(len(train_labels))

25000


We now have a train and test dataset, but let’s also also create a validation set which we can use for for evaluation and tuning without tainting our test set results. Sklearn has a convenient utility for creating such splits:

In [46]:
from sklearn.model_selection import train_test_split
train_text, val_texts, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=.2)

Alright, we’ve read in our dataset. Now let’s tackle tokenization. We’ll eventually train a classifier using pre-trained DistilBert, so let’s use the DistilBert tokenizer.

In [49]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Now we can simply pass our texts to the tokenizer. We’ll pass truncation=True and padding=True, which will ensure that all of our sequences are padded to the same length and are truncated to be no longer model’s maximum input length. This will allow us to feed batches of sequences into the model at the same time.

In [52]:
train_encodings = tokenizer(train_text, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)

Now, let’s turn our labels and encodings into a Dataset object. In PyTorch, this is done by subclassing a torch.utils.data.Dataset object and implementing __len__ and __getitem__. In TensorFlow, we pass our input encodings and labels to the from_tensor_slices constructor method. We put the data in this format so that the data can be easily batched such that each key in the batch encoding corresponds to a named parameter of the forward() method of the model we will train.

In [53]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

Finetune with HuggingFace Trainer

In [57]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss
10,0.693800
20,0.690500
30,0.689800
40,0.687900
50,0.682400
60,0.679800
70,0.652400
80,0.617200
90,0.555400
100,0.427300


TrainOutput(global_step=3750, training_loss=0.17730890169590713, metrics={'train_runtime': 6743.4226, 'train_samples_per_second': 0.556, 'total_flos': 12341147443200000, 'epoch': 3.0})

after training

In [59]:
trainer.save_model("./ImdbNBert")

In [62]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis", model="./ImdbNBert",tokenizer=tokenizer )

In [63]:
sentiment_analysis("It has been a good day")

[{'label': 'LABEL_1', 'score': 0.9978824853897095}]

In [64]:
sentiment_analysis("I never liked her anyways")

[{'label': 'LABEL_0', 'score': 0.9892675280570984}]